In [ ]:
import numpy as np
import polars as pl
import pandas as pd
import catboost as cbt
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import gc
import warnings

# Настройки
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [ ]:
class Config:
    target = "responder_6"
    features = [f"feature_{i:02d}" for i in range(79)] 
    lags = [f"responder_{i}_lag_1" for i in range(9)]
    cat_features = ["symbol_id"]
    val_size = 0.1
    n_splits = 2  # Можно уменьшить до 1 для максимальной экономии памяти

train_path = "/kaggle/input/Preprocessing/training.parquet/"
valid_path = "/kaggle/input/Preprocessing/validation.parquet/"

In [ ]:
def load_data():
    """Оптимизированная загрузка данных"""
    columns = Config.features + Config.lags + Config.cat_features + [Config.target, "weight"]
    return (
        pl.scan_parquet(train_path)
        .select(columns)
    )

def prepare_features(data):
    """Подготовка фичей с контролем памяти"""
    df = data.collect(streaming=True)
    # Переводим все фичи в float32 для экономии памяти
    X = df[Config.features + Config.lags + Config.cat_features].to_pandas()
    X = X.astype({col: np.float32 for col in Config.features + Config.lags})
    y = df[Config.target].to_numpy().astype(np.float32)
    weights = df["weight"].to_numpy().astype(np.float32)

    del df
    gc.collect()
    for col in X.columns:
        if X[col].isnull().any():
            X[col].fillna(X[col].mean(), inplace=True)
    cat_col_idx = [X.columns.get_loc(col) for col in Config.cat_features]
    X[Config.cat_features] = X[Config.cat_features].astype('category')
    return X, y, weights, cat_col_idx

def weighted_r2(y_true, y_pred, weights):
    """Эффективная реализация взвешенной R2"""
    weights = np.ones_like(y_true) if weights is None else weights
    y_mean = np.average(y_true, weights=weights)
    numerator = np.sum(weights * (y_true - y_pred)**2)
    denominator = np.sum(weights * (y_true - y_mean)**2)
    return 1 - numerator / (denominator + 1e-10)

In [ ]:
def train_xgboost(X, y, weights, n_splits=3):
    """Обучение XGBoost с визуализацией реальных и предсказанных значений на каждом фолде"""
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []

    cat_col = Config.cat_features[0]
    X_encoded = pd.get_dummies(X, columns=[cat_col], dtype=np.float32)

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_encoded), 1):
        print(f"\nXGBoost Fold {fold}/{n_splits}")

        X_train, X_val = X_encoded.iloc[train_idx], X_encoded.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        w_train, w_val = weights[train_idx], weights[val_idx]

        X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
        X_train = X_train.astype(np.float32)
        X_val = X_val.astype(np.float32)

        model = xgb.XGBRegressor(
            objective='reg:squarederror',
            learning_rate=0.05,
            n_estimators=200,
            max_depth=3,
            subsample=0.5,
            colsample_bytree=0.5,
            min_child_weight=10,
            tree_method='gpu_hist',
            predictor='gpu_predictor',
            eval_metric='rmse',
            n_jobs=1
        )

        model.fit(
            X_train, y_train,
            sample_weight=w_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            sample_weight_eval_set=[w_train, w_val],
            verbose=100,
            early_stopping_rounds=10
        )

        y_pred = model.predict(X_val)
        score = weighted_r2(y_val, y_pred, w_val)
        results.append(score)
        print(f"XGBoost Fold {fold} R2: {score:.4f}")

        # Визуализация реальных и предсказанных значений
        plt.figure(figsize=(12, 5))
        plt.plot(y_val, label='Real', alpha=0.7)
        plt.plot(y_pred, label='Predicted', alpha=0.7)
        plt.title(f'Fold {fold} - Real vs Predicted')
        plt.xlabel('Sample index')
        plt.ylabel('Target value')
        plt.legend()
        plt.grid(True)
        plt.show()

        del model, X_train, X_val, y_train, y_val, w_train, w_val
        gc.collect()

    del X_encoded
    gc.collect()
    return results

def train_lightgbm(X, y, weights, cat_col_idx, n_splits=3):
    """Обучение LightGBM"""
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
        print(f"\nLightGBM Fold {fold}/{n_splits}")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        w_train, w_val = weights[train_idx], weights[val_idx]
        # Используем float32
        X_train = X_train.astype(np.float32)
        X_val = X_val.astype(np.float32)
        model = lgb.LGBMRegressor(
            objective="regression",
            n_estimators=200,      # Меньше деревьев
            learning_rate=0.05,
            max_depth=3,
            num_leaves=15,
            min_child_samples=50,
            subsample=0.5,
            colsample_bytree=0.5,
            device='gpu',
            verbose=-1
        )
        model.fit(
            X_train, y_train,
            sample_weight=w_train,
            eval_set=[(X_val, y_val)],
            eval_sample_weight=[w_val],
            eval_metric='rmse',
            categorical_feature=cat_col_idx,
            callbacks=[lgb.early_stopping(10), lgb.log_evaluation(20)]
        )
        score = weighted_r2(y_val, model.predict(X_val), w_val)
        results.append(score)
        print(f"LightGBM Fold {fold} R2: {score:.4f}")
        del model, X_train, X_val, y_train, y_val, w_train, w_val
        gc.collect()
    return results

def train_catboost(X, y, weights, cat_col_idx, n_splits=3):
    """Обучение CatBoost"""
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []
    
    # Преобразование категориального признака
    X_cb = X.copy()
    cat_col = X.columns[cat_col_idx[0]]  
    X_cb[cat_col] = X_cb[cat_col].astype(int).astype(str)
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_cb), 1):
        print(f"\nCatBoost Fold {fold}/{n_splits}")
        
        X_train, X_val = X_cb.iloc[train_idx], X_cb.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        w_train, w_val = weights[train_idx], weights[val_idx]
        
        model = cbt.CatBoostRegressor(
            iterations=200,
            learning_rate=0.05,
            loss_function='RMSE',
            eval_metric='R2',
            early_stopping_rounds=20,
            cat_features=cat_col_idx,
            task_type='GPU',
            verbose=100
        )
        
        model.fit(
            X_train, y_train,
            sample_weight=w_train,
            eval_set=(X_val, y_val)
        )
        
        score = weighted_r2(y_val, model.predict(X_val), w_val)
        results.append(score)
        print(f"CatBoost Fold {fold} R2: {score:.4f}")
        
        del model, X_train, X_val, y_train, y_val, w_train, w_val
        gc.collect()
    
    return results



"""Сравнение моделей"""
# print("Loading data...")
data = load_data()
X, y, weights, cat_col_idx = prepare_features(data)
del data
gc.collect()
print("\n=== Training LightGBM ===")
lgb_results = train_lightgbm(X, y, weights, cat_col_idx, n_splits=3)
print("\n=== Training CatBoost ===")
cb_results = train_catboost(X, y, weights, cat_col_idx, n_splits=3)
print("\n=== Training XGBoost ===")
xgb_results = train_xgboost(X, y, weights)

results = {
    "XGBoost": xgb_results,
    "LightGBM": lgb_results,
    "CatBoost": cb_results
}


In [ ]:
plt.figure(figsize=(10, 5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for (model, scores), color in zip(results.items(), colors):
    plt.plot(range(1, len(scores)+1), scores, 
            label=model, marker='o', linewidth=2, color=color)
plt.title("Model Comparison by Fold (Weighted R2)", fontsize=14)
plt.xlabel("Fold Number", fontsize=12)
plt.ylabel("Weighted R2 Score", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(range(1, 2))
plt.tight_layout()
plt.show()
print("\n=== Final Results ===")
final_scores = {}
for model, scores in results.items():
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    final_scores[model] = mean_score
    print(f"{model}:")
    print(f"  Mean R2: {mean_score:.4f}")
    print(f"  Std R2: {std_score:.4f}")
    print(f"  Fold scores: {[round(s, 4) for s in scores]}")
best_model = max(final_scores.items(), key=lambda x: x[1])
print(f"\nRecommendation: Use {best_model[0]} as baseline (highest mean R2: {best_model[1]:.4f})")